In [ ]:
import pandas as pd
import numpy as np

# Path to your activation data file
file_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\conv1_average_activation_matrix.csv'

# Load the data
data = pd.read_csv(file_path)

# Perform SVD
U, S, Vt = np.linalg.svd(data, full_matrices=False)

# Set the number of components to retain (e.g., 100)
n_components = 100

# Select the top components
U_reduced = U[:, :n_components]
S_reduced = np.diag(S[:n_components])
Vt_reduced = Vt[:n_components, :]

# Form the reduced matrix
reduced_data = np.dot(U_reduced, S_reduced)

# Save the reduced data
output_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\conv1_svd_100_components.csv'
pd.DataFrame(reduced_data).to_csv(output_path, index=False)

print(f"SVD completed for conv1, saved to {output_path}")


In [ ]:
# Calculate total variance
total_variance = np.sum(S**2)

# Calculate explained variance for the top 100 components
n_components = 100
explained_variance = np.sum(S[:n_components]**2)

# Calculate the explained variance ratio
explained_variance_ratio = explained_variance / total_variance
print(f"Explained Variance Ratio for top {n_components} components: {explained_variance_ratio:.2%}")

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming 'pca' is the PCA model used earlier
# and 'kmeans' is the trained KMeans model
# Load original activation matrix
layer_file_path = '/home/tpolklabuser/Desktop/Violet/DNN/ML_RR/DNN_Activation/conv1_average_activation_matrix.csv'
activation_matrix = np.loadtxt(layer_file_path, delimiter=',', skiprows=1)  # Shape: (710016, 12)

# Step 1: Apply the same PCA transformation to the activation matrix
activation_matrix_pca = pca.transform(activation_matrix)  # Shape: (710016, 10)

# Step 2: Predict the closest centroids for each unit
closest_centroids = kmeans.predict(activation_matrix_pca)

# Step 3: Reconstruct the activation matrix using the closest centroids
reconstructed_matrix_pca = kmeans.cluster_centers_[closest_centroids]  # Shape: (710016, 10)

# Step 4: Inverse transform to get back to the original feature space
reconstructed_matrix = pca.inverse_transform(reconstructed_matrix_pca)  # Shape: (710016, 12)

# Step 5: Calculate the mean squared reconstruction error
reconstruction_error = mean_squared_error(activation_matrix, reconstructed_matrix)
print(f"Reconstruction Error (MSE): {reconstruction_error:.4f}")

# Step 6: Calculate explained variance as a measure of clustering performance
total_variance = np.var(activation_matrix)
explained_variance_ratio = 1 - (reconstruction_error / total_variance)
print(f"Explained Variance by Clustering: {explained_variance_ratio:.2%}")


In [8]:
# This codes using SVD instead of PCA, according to Justine Zhang report, 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD

# Load or create your activation matrix
file = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\pool1_both_combined_matrix.csv'
activation_matrix = np.loadtxt(file, delimiter=',', skiprows=1) # Example loading step

# transform to .npy for futher use
# np.save('conv1_both_combined_matrix.npy', activation_matrix)

# Define the number of components to keep (adjust based on the desired explained variance)
n_components = 90  # Choose an appropriate number based on variance explained or experiment needs

activation_matrix_centered = activation_matrix - np.mean(activation_matrix, axis=0)

scaler = StandardScaler()
activation_matrix_standardized = scaler.fit_transform(activation_matrix_centered)

# Transpose the matrix to reduce along the units dimension (710061)
activation_matrix_T = activation_matrix_standardized.T  # Now shape is [120, 710061]


# Perform Truncated SVD
svd = TruncatedSVD(n_components=n_components)
reduced_matrix = svd.fit_transform(activation_matrix_T)


# Transpose back to retain the structure where rows represent units
reduced_matrix = reduced_matrix.T  # Final shape will be [n_components, 120]

# Print the shape of the reduced matrix to verify dimensions
print("Reduced matrix shape:", reduced_matrix.shape)

# Print the explained variance ratio to understand how much of the original data is captured in the reduced matrix.
print("Explained variance ratio:", svd.explained_variance_ratio_.sum())

# Optional: Inspect the singular values, which represent the importance of each reduced dimension.
# High singular values correlate with high variance directions, often capturing meaningful structures in the data.
singular_values = svd.singular_values_
print("Singular values:", singular_values)

# Save the reduced matrix for future analysis or model input
# np.save('reduced_activation_matrix.npy', reduced_matrix)

# np.save('C:\\Users\\quanz\\Documents\\UM\\Projects\\DNN_in_Neural_Dedifferentiation\\SVD_results\\reduced_activation_matrix_rnorm1.npy', reduced_matrix)


Reduced matrix shape: (90, 120)
Explained variance ratio: 0.875526576491034
Singular values: [3245.24257252  958.93232008  620.28393087  469.41932077  454.65545033
  446.5940917   437.53825277  420.88500846  402.84125898  399.47392089
  397.45453855  383.66613994  380.2149005   376.26052622  370.11725558
  369.03738669  367.30934737  360.08292225  353.53069113  352.5294927
  347.85151518  345.83458232  343.18314249  338.92800378  335.34113844
  332.95450035  330.94591394  329.62420034  324.50480316  322.13952192
  319.30831705  315.03322686  311.064738    309.56895758  306.29250633
  305.36092273  303.32890128  302.64255447  300.41460433  298.03028279
  297.33561466  292.18356948  291.1143555   289.52058736  287.93567337
  284.77125787  282.89409387  282.49132802  282.00831347  278.93800197
  277.44486691  276.20929582  274.94715148  273.82652535  271.26426614
  270.26298475  268.05677778  267.36127764  266.35090124  265.34048197
  262.9089305   260.61958926  258.99714297  258.64088345

In [12]:
import numpy as np

# Load the reduced activation matrix from the file
file_path = r'C:\Users\quanz\Documents\UM\Projects\DNN_in_Neural_Dedifferentiation\SVD_results\reduced_activation_matrix_rnorm1.npy'
reduced_activation_matrix = np.load(file_path)

# Print the shape of the loaded matrix
print(reduced_activation_matrix.shape)


(90, 120)


In [ ]:
# This code will average the SVD matrix across time domain, so make the output from the previous chunck [100x120] to [100x6] condition-wise to prepare for risdge regression
import numpy as np
import glob

# Load all .npy files
file_paths = glob.glob(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\reduced_activation_matrix_*.npy')  # Adjust path as needed
activation_matrices = {file.split('\\')[-1].split('.')[0]: np.load(file) for file in file_paths}

# Function to average every 10 columns
def average_columns(matrix, group_size=10):
    return matrix.reshape(matrix.shape[0], -1, group_size).mean(axis=2)

# Apply column averaging to each matrix
reduced_matrices = {}
for name, matrix in activation_matrices.items():
    reduced_matrix = average_columns(matrix, group_size=10)  # Shape will be (100, 12)
    reduced_matrices[name] = reduced_matrix

# Optional: Save reduced matrices if you need them for later
for name, reduced_matrix in reduced_matrices.items():
    np.save(f"C:\\Users\\quanz\\Documents\\UM\\Projects\\GLX_Project\\DNN\\MiND_Stimili\\reduced_{name}.npy", reduced_matrix)



In [16]:
# This code help update the average across time matrix for a specific layer
import numpy as np

# Specify the file path for conv3
file_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\reduced_activation_matrix_conv5_G.npy'

# Load the specific file
activation_matrix = np.load(file_path)

# Function to average every 10 columns
def average_columns(matrix, group_size=10):
    return matrix.reshape(matrix.shape[0], -1, group_size).mean(axis=2)

# Apply column averaging
reduced_matrix = average_columns(activation_matrix, group_size=10)  # Shape will be (100, 12)

# Save the reduced matrix
output_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\reduced_reduced_activation_matrix_conv5_G.npy'
np.save(output_path, reduced_matrix)

print(f"Reduced matrix saved to: {output_path}")


Reduced matrix saved to: C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\reduced_reduced_activation_matrix_conv5_G.npy


In [6]:
print(name)


NameError: name 'name' is not defined

In [ ]:
# To see how much variance it explaines: 
total_variance_explained = svd.explained_variance_ratio_.sum()
print("Total variance explained:", total_variance_explained)

import matplotlib.pyplot as plt

cumulative_variance = np.cumsum(svd.explained_variance_ratio_)
plt.plot(cumulative_variance)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by Number of Components')
plt.show()